## Passo 1: Importação das Bibliotecas

In [1]:
import pandas as pd

## Passo 2: Carregamento do Dataset

In [2]:
arquivo_csv = './../DADOS/dados_combinados.csv'  
dados = pd.read_csv(arquivo_csv, sep=',')

## Passo 3: Visualização Inicial do Dataset

In [3]:
print("Visualização inicial do dataset:")
print(dados.head())

Visualização inicial do dataset:
   ID_COMODO  ID_USUARIO NOME_COMODO                      DESCRICAO_x  \
0          1           1     Varanda             CAA modo 1 - arejado   
1          2           2      Quarto  CAA modo 2 - com mAA3veis novos   
2          3           3     Varanda             CAA modo 3 - arejado   
3          4           4      Quarto       CAA modo 4 - bem iluminado   
4          5           5      Quarto  CAA modo 5 - com mAA3veis novos   

   ID_CONSUMO_MENSAL       MES_REFERENCIA  CONSUMO_MENSAL_KWH  \
0                1.0  2024-02-01 00:00:00              540.99   
1                2.0  2024-03-01 00:00:00              414.30   
2                3.0  2024-04-01 00:00:00              668.66   
3                4.0  2024-05-01 00:00:00              932.57   
4                5.0  2024-06-01 00:00:00              284.18   

   ID_ELETRODOMESTICO NOME_ELETRODOMESTICO  POTENCIA_WATTS  ...         CEP  \
0                   1      Ar-condicionado            1662

## Passo 4: Exibição de Informações Gerais

In [4]:
print("\nInformações gerais:")
print(dados.info())


Informações gerais:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID_COMODO             49 non-null     int64  
 1   ID_USUARIO            49 non-null     int64  
 2   NOME_COMODO           49 non-null     object 
 3   DESCRICAO_x           49 non-null     object 
 4   ID_CONSUMO_MENSAL     38 non-null     float64
 5   MES_REFERENCIA        38 non-null     object 
 6   CONSUMO_MENSAL_KWH    38 non-null     float64
 7   ID_ELETRODOMESTICO    49 non-null     int64  
 8   NOME_ELETRODOMESTICO  49 non-null     object 
 9   POTENCIA_WATTS        49 non-null     int64  
 10  HORAS_USO_DIA         49 non-null     int64  
 11  DESCRICAO_y           49 non-null     object 
 12  ID_ENDERECO           49 non-null     int64  
 13  LOGRADOURO            49 non-null     object 
 14  NUMERO                49 non-null     int64  
 15  COMP

## Passo 5: Estatísticas Descritivas

In [5]:
print("\nEstatísticas descritivas:")
print(dados.describe())


Estatísticas descritivas:
       ID_COMODO  ID_USUARIO  ID_CONSUMO_MENSAL  CONSUMO_MENSAL_KWH  \
count  49.000000   49.000000          38.000000           38.000000   
mean   25.551020   25.551020          24.710526          566.905526   
std    14.723933   14.723933          14.837695          253.351337   
min     1.000000    1.000000           1.000000          111.640000   
25%    13.000000   13.000000          13.250000          361.992500   
50%    26.000000   26.000000          25.500000          611.090000   
75%    38.000000   38.000000          37.750000          809.372500   
max    50.000000   50.000000          50.000000          964.770000   

       ID_ELETRODOMESTICO  POTENCIA_WATTS  HORAS_USO_DIA  ID_ENDERECO  \
count           49.000000       49.000000      49.000000    49.000000   
mean            25.551020     1708.877551       4.795918    25.551020   
std             14.723933      682.256814       3.068520    14.723933   
min              1.000000      630.000000

## Passo 6: Seleção de Colunas Relevantes

In [6]:
colunas_relevantes = ['CONSUMO_MENSAL_KWH', 'POTENCIA_WATTS', 'HORAS_USO_DIA', 'NOME_ELETRODOMESTICO']
dados_filtrados = dados[colunas_relevantes]

## Passo 7: Verificação de Valores Ausentes

In [7]:
print("Dados filtrados:")
print(dados_filtrados.head())

Dados filtrados:
   CONSUMO_MENSAL_KWH  POTENCIA_WATTS  HORAS_USO_DIA NOME_ELETRODOMESTICO
0              540.99            1662              1      Ar-condicionado
1              414.30            2850              7             LAAmpada
2              668.66             900              5               FogAAo
3              932.57            1152              2                   TV
4              284.18            1427              7            Geladeira


## Passo 8: Tratamento de Valores Ausentes

In [8]:
print("\nValores ausentes:")
print(dados_filtrados.isnull().sum())


Valores ausentes:
CONSUMO_MENSAL_KWH      11
POTENCIA_WATTS           0
HORAS_USO_DIA            0
NOME_ELETRODOMESTICO     0
dtype: int64


In [9]:
mediana_consumo = dados_filtrados['CONSUMO_MENSAL_KWH'].median()
dados_filtrados['CONSUMO_MENSAL_KWH'].fillna(mediana_consumo, inplace=True)

print("Valores ausentes após preenchimento:")
print(dados_filtrados.isnull().sum())

print("\nDados filtrados (amostra):")
print(dados_filtrados.head())


Valores ausentes após preenchimento:
CONSUMO_MENSAL_KWH      0
POTENCIA_WATTS          0
HORAS_USO_DIA           0
NOME_ELETRODOMESTICO    0
dtype: int64

Dados filtrados (amostra):
   CONSUMO_MENSAL_KWH  POTENCIA_WATTS  HORAS_USO_DIA NOME_ELETRODOMESTICO
0              540.99            1662              1      Ar-condicionado
1              414.30            2850              7             LAAmpada
2              668.66             900              5               FogAAo
3              932.57            1152              2                   TV
4              284.18            1427              7            Geladeira


C:\Users\zenet\AppData\Local\Temp\ipykernel_42256\3305409015.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dados_filtrados['CONSUMO_MENSAL_KWH'].fillna(mediana_consumo, inplace=True)
C:\Users\zenet\AppData\Local\Temp\ipykernel_42256\3305409015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_filtrados['CONSUMO_MENSAL_KWH'].filln

## Passo 9: Categorização dos Dados

In [10]:
limiar_consumo = dados_filtrados['CONSUMO_MENSAL_KWH'].median()

dados_filtrados['CATEGORIA_CONSUMO'] = dados_filtrados['CONSUMO_MENSAL_KWH'].apply(
    lambda x: 'Eficiente' if x <= limiar_consumo else 'Ineficiente'
)

print("\nDados categorizados (amostra):")
print(dados_filtrados[['CONSUMO_MENSAL_KWH', 'CATEGORIA_CONSUMO']].head())



Dados categorizados (amostra):
   CONSUMO_MENSAL_KWH CATEGORIA_CONSUMO
0              540.99         Eficiente
1              414.30         Eficiente
2              668.66       Ineficiente
3              932.57       Ineficiente
4              284.18         Eficiente


C:\Users\zenet\AppData\Local\Temp\ipykernel_42256\1081636530.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados_filtrados['CATEGORIA_CONSUMO'] = dados_filtrados['CONSUMO_MENSAL_KWH'].apply(


In [11]:
import plotly.express as px

contagem_categorias = dados_filtrados['CATEGORIA_CONSUMO'].value_counts()

fig = px.pie(
    values=contagem_categorias.values,
    names=contagem_categorias.index,
    title='Distribuição de Categorias de Consumo',
    hole=0.3  
)

fig.show()